In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
from census import Census

In [2]:
# Census API key
from courtCensusKey import censusKey

In [3]:
complete_url='https://api.census.gov/data/timeseries/intltrade/exports/hs'+\
            '?get=DISTRICT,DIST_NAME,ALL_VAL_MO,ALL_VAL_YR,E_COMMODITY_SDESC'+\
            '&E_COMMODITY=0901'+\
            '&YEAR=2019'+\
            '&MONTH=*'

In [4]:
response=requests.get(complete_url)
response

<Response [200]>

In [5]:
pd.DataFrame(response.json())

,0,1,2,3,4,5,6,7
0,DISTRICT,DIST_NAME,ALL_VAL_MO,ALL_VAL_YR,E_COMMODITY_SDESC,E_COMMODITY,YEAR,MONTH
1,-,TOTAL FOR ALL DISTRICTS,68597909,68597909,COFFEE; COFFEE HUSKS ETC; SUBSTITUTES WITH COFFEE,0901,2019,01
2,07,"OGDENSBURG, NY",5385716,5385716,COFFEE; COFFEE HUSKS ETC; SUBSTITUTES WITH COFFEE,0901,2019,01
3,01,"PORTLAND, ME",9343,9343,COFFEE; COFFEE HUSKS ETC; SUBSTITUTES WITH COFFEE,0901,2019,01
4,02,"ST. ALBANS, VT",464023,464023,COFFEE; COFFEE HUSKS ETC; SUBSTITUTES WITH COFFEE,0901,2019,01
...,...,...,...,...,...,...,...,...
427,52,"MIAMI, FL",1215633,16091751,COFFEE; COFFEE HUSKS ETC; SUBSTITUTES WITH COFFEE,0901,2019,12
428,53,"HOUSTON-GALVESTON, TX",188477,4676030,COFFEE; COFFEE HUSKS ETC; SUBSTITUTES WITH COFFEE,0901,2019,12
429,54,"WASHINGTON, DC",0,43078,COFFEE; COFFEE HUSKS ETC; SUBSTITUTES WITH COFFEE,0901,2019,12
430,55,"DALLAS-FORT WORTH, TX",0,87361,COFFEE; COFFEE HUSKS ETC; SUBSTITUTES WITH COFFEE,0901,2019,12


In [6]:
#Setting variables 
year='2019'
month='*'
cols='DISTRICT,DIST_NAME,ALL_VAL_MO,ALL_VAL_YR,E_COMMODITY_SDESC'


In [26]:
#Base URL 
base_url = 'https://api.census.gov/data/timeseries/intltrade/exports/hs'

complete_url = f'{base_url}?get={cols}&YEAR={year}&MONTH={month}'

# HTC Codes and what they represent
# 0901 - Coffee
# 480300 - Toilet Paper
# 2203 - Beer
# 2204 - Wine
# Hard alcohol - 2207

# Items in US top Exports
# Medical Equipment - $83 billion
# Plastics - $60 billion
# Pharmaceuticals - $47 billion
    # 3002 - Vaccines, Blood, and Immunological products
    # 3003 - Medication (unmeasured doses)
    # 3004 - Medication (measured doses)
    # 3005 - Medically-treated Bandages
    # 3006 - Surgical Equipment
# Organic chemicals - $39 billion

hts_list=['3002', '3003', '3004', '3005', '3006']
df_list=[]

for each_hts in hts_list:
    hts_code=each_hts
    query_params={
        'get': 'CTY_CODE,CTY_NAME,DF,DISTRICT,DIST_NAME,ALL_VAL_MO,ALL_VAL_YR,E_COMMODITY_SDESC',
        'E_COMMODITY': hts_code,
        'YEAR': '2019',
        'MONTH': "*"
    }
    max_response=requests.get(base_url, params=query_params)
    #print(max_response.json())
    df_list.append(max_response.json()[1:])
#print(df_list)

In [27]:
productList = []
for product in df_list:
    productList += product

In [28]:
exportData = pd.DataFrame(productList, columns=max_response.json()[0])
exportData

,CTY_CODE,CTY_NAME,DF,DISTRICT,DIST_NAME,ALL_VAL_MO,ALL_VAL_YR,E_COMMODITY_SDESC,E_COMMODITY,YEAR,MONTH
0,-,TOTAL FOR ALL COUNTRIES,-,01,"PORTLAND, ME",239187,239187,HUMAN BLOOD; ANIMAL BLOO,3002,2019,01
1,-,TOTAL FOR ALL COUNTRIES,-,02,"ST. ALBANS, VT",3566,3566,HUMAN BLOOD; ANIMAL BLOO,3002,2019,01
2,-,TOTAL FOR ALL COUNTRIES,-,04,"BOSTON, MA",45953630,45953630,HUMAN BLOOD; ANIMAL BLOO,3002,2019,01
3,-,TOTAL FOR ALL COUNTRIES,-,07,"OGDENSBURG, NY",18872913,18872913,HUMAN BLOOD; ANIMAL BLOO,3002,2019,01
4,-,TOTAL FOR ALL COUNTRIES,-,09,"BUFFALO, NY",27631834,27631834,HUMAN BLOOD; ANIMAL BLOO,3002,2019,01
...,...,...,...,...,...,...,...,...,...,...,...
219226,4XXX,EUROPE,1,80,MAIL SHIPMENTS,12878,28031,PHARMACEUTICAL GOODS IN NOTE 4 TO CHAPTER 30,3006,2019,12
219227,5880,JAPAN,1,80,MAIL SHIPMENTS,0,3285,PHARMACEUTICAL GOODS IN NOTE 4 TO CHAPTER 30,3006,2019,12
219228,5XXX,ASIA,1,80,MAIL SHIPMENTS,0,3285,PHARMACEUTICAL GOODS IN NOTE 4 TO CHAPTER 30,3006,2019,12
219229,7850,MAURITIUS,2,80,MAIL SHIPMENTS,0,4900,PHARMACEUTICAL GOODS IN NOTE 4 TO CHAPTER 30,3006,2019,12


In [29]:
exportData['E_COMMODITY_SDESC'].unique()

array(['HUMAN BLOOD; ANIMAL BLOO', 'MEDICAMENTS NESOI OF MIXTU',
       'MEDICAMENTS NESOI, MIXED O', 'BANDAGES ETC COATED ETC OR',
       'PHARMACEUTICAL GOODS IN NOTE 4 TO CHAPTER 30'], dtype=object)

In [11]:
#Convert to country data to dataframe, perform some basic operations
# country_df = pd.DataFrame(countryData[1:], columns = countryData[0]).\
exportData = exportData.rename(columns={"CTY_CODE": "COUNTRY_CODE", "CTY_NAME": "COUNTRY_NAME", "DF": "DOMESTIC_FOREIGN_CODE", "DIST_NAME": "DISTRICT_NAME", "ALL_VAL_MO": "ALL_VALUES_MONTH", "ALL_VAL_YR": "ALL_VALUES_YEAR", "E_COMMODITY_SDESC": "COMMODITY_DESCRIPTION", "E_COMMODITY": "COMMODITY"})
exportData
# country_df['fips']=country_df.state+country_df.country
# country_df.set_index('fips',inplace=True)
# country_df.drop(columns=['state','country'],inplace=True)
# country_df[['Country','State']] = county_df.CountryST.str.split(",",expand=True)
# country_df.dropna(how="any", inplace=True)
# country_df

,COUNTRY_CODE,COUNTRY_NAME,DOMESTIC_FOREIGN_CODE,DISTRICT,DISTRICT_NAME,ALL_VALUES_MONTH,ALL_VALUES_YEAR,COMMODITY_DESCRIPTION,COMMODITY,YEAR,MONTH
0,-,TOTAL FOR ALL COUNTRIES,-,-,TOTAL FOR ALL DISTRICTS,68597909,68597909,COFFEE; COFFEE HUSKS ETC; SUBSTITUTES WITH COFFEE,0901,2019,01
1,-,TOTAL FOR ALL COUNTRIES,1,01,"PORTLAND, ME",4761,4761,COFFEE; COFFEE HUSKS ETC; SUBSTITUTES WITH COFFEE,0901,2019,01
2,-,TOTAL FOR ALL COUNTRIES,2,01,"PORTLAND, ME",4582,4582,COFFEE; COFFEE HUSKS ETC; SUBSTITUTES WITH COFFEE,0901,2019,01
3,-,TOTAL FOR ALL COUNTRIES,1,02,"ST. ALBANS, VT",107553,107553,COFFEE; COFFEE HUSKS ETC; SUBSTITUTES WITH COFFEE,0901,2019,01
4,-,TOTAL FOR ALL COUNTRIES,2,02,"ST. ALBANS, VT",356470,356470,COFFEE; COFFEE HUSKS ETC; SUBSTITUTES WITH COFFEE,0901,2019,01
...,...,...,...,...,...,...,...,...,...,...,...
24079,7620,ANGOLA,1,-,TOTAL FOR ALL DISTRICTS,0,28363,COFFEE; COFFEE HUSKS ETC; SUBSTITUTES WITH COFFEE,0901,2019,12
24080,7650,LIBERIA,1,-,TOTAL FOR ALL DISTRICTS,0,2606,COFFEE; COFFEE HUSKS ETC; SUBSTITUTES WITH COFFEE,0901,2019,12
24081,7910,SOUTH AFRICA,1,-,TOTAL FOR ALL DISTRICTS,0,3195,COFFEE; COFFEE HUSKS ETC; SUBSTITUTES WITH COFFEE,0901,2019,12
24082,7XXX,AFRICA,1,-,TOTAL FOR ALL DISTRICTS,20420,476008,COFFEE; COFFEE HUSKS ETC; SUBSTITUTES WITH COFFEE,0901,2019,12


In [12]:
exportData = exportData[exportData['COUNTRY_NAME'] != "TOTAL FOR ALL COUNTRIES"]
exportData = exportData[exportData['DISTRICT_NAME'] != "TOTAL FOR ALL DISTRICTS"]
exportData

,COUNTRY_CODE,COUNTRY_NAME,DOMESTIC_FOREIGN_CODE,DISTRICT,DISTRICT_NAME,ALL_VALUES_MONTH,ALL_VALUES_YEAR,COMMODITY_DESCRIPTION,COMMODITY,YEAR,MONTH
302,0020,NAFTA,-,01,"PORTLAND, ME",9343,9343,COFFEE; COFFEE HUSKS ETC; SUBSTITUTES WITH COFFEE,0901,2019,01
303,0022,OECD,-,01,"PORTLAND, ME",9343,9343,COFFEE; COFFEE HUSKS ETC; SUBSTITUTES WITH COFFEE,0901,2019,01
304,0023,NATO,-,01,"PORTLAND, ME",9343,9343,COFFEE; COFFEE HUSKS ETC; SUBSTITUTES WITH COFFEE,0901,2019,01
305,0026,APEC,-,01,"PORTLAND, ME",9343,9343,COFFEE; COFFEE HUSKS ETC; SUBSTITUTES WITH COFFEE,0901,2019,01
306,1220,CANADA,-,01,"PORTLAND, ME",9343,9343,COFFEE; COFFEE HUSKS ETC; SUBSTITUTES WITH COFFEE,0901,2019,01
...,...,...,...,...,...,...,...,...,...,...,...
23759,0026,APEC,2,70,LOW VALUE,287240,7112998,COFFEE; COFFEE HUSKS ETC; SUBSTITUTES WITH COFFEE,0901,2019,12
23760,1220,CANADA,1,70,LOW VALUE,7213661,78329705,COFFEE; COFFEE HUSKS ETC; SUBSTITUTES WITH COFFEE,0901,2019,12
23761,1220,CANADA,2,70,LOW VALUE,287240,7112998,COFFEE; COFFEE HUSKS ETC; SUBSTITUTES WITH COFFEE,0901,2019,12
23762,1XXX,NORTH AMERICA,1,70,LOW VALUE,7213661,78329705,COFFEE; COFFEE HUSKS ETC; SUBSTITUTES WITH COFFEE,0901,2019,12


In [13]:
# Save as a csv
# Using encoding="utf-8" to avoid issues later
exportData.to_csv("Resources/exportData2019.csv", encoding="utf-8", index=False)